# Mixed Effects Regression Models for Continuous Outcomes

Limitations of ANOVA:

    Compound symetry assumption and must have complete data 
    
MANOVA:

    Compound symetry assumption, must have complete data and describes group trends over
    time, not individual's effect of trend.
    
### Mixed Effects Regression Models (MRMs)

MRMs include random-effects in a regression model to account for an indivisual's influence on ther reppeated observation, dicribing each subjects trend accross time. They also indicate the degree of subject variation that exists in the population of subjects.

Useful Because:

    1) Subject not assumed to be measured all at the same time, easier to handle 
    missing data
        - increases the statistical power and reduced missing data bias
    2) ANOVA and MANOVA estimates the average change across time, MRMs estimates the
    average change and individual change across time
    
This notebook will expand on a Simple Linear Regression model to include random effects

### Simple Linear Regression Model

$$y_{ij} = \beta_0 + \beta_1 t_{ij} + \epsilon_{ij}$$
where 

$i = 1, 2, ..., N$ Subjects

for

$j = 1, 2, ..., n_i$ Occasions

Note:

$y_{ij}, t_{ij}, \epsilon_{ij}$ all have the subscripts $ij$. Therefore $y, t, \epsilon$ are allowed be different for each individual and ocassion.

### Adding Random Intecept $v$

To allow for the influence of each individual on their repeated measure we can add the $v_{0i}$ term to represent the influence of an individual $i$ on their repeated observation:

$$y_{ij} = v_{0i} + \beta_0 + \beta_1 t_{ij} + \epsilon_{ij}$$

for organization purposes:

$$y_{ij} = \beta_0 + \beta_1 t_{ij} + v_{0i} + \epsilon_{ij}$$

We can represent the model as a hierarchical or multilevel model:

#### Level 1 (within-subject effect):
$$y_{ij} = b_{0i} + b_{1i} t_{ij} + \epsilon_{ij}$$

Representing that an individual $i$'s response, at time $j$, is influenced by the individual's initial level $b_{0i}$ and time trend $b_{1i}$


#### Level 2 (between-subject effect):

$$b_{0i} = \beta_0 + v_{0i}$$
$$b_{1i} = \beta_1$$

Representing an individual $i$'s initial level is determined by the population's initial level $\beta_0$ and his/her individuals diviations from the population, represented by $v_{0i}$. This model also states that each individual has the same slope, that is parallel to the population's slope, expressed by $b_{1i} = \beta_1$.



Note on incomplete data:

The underlying assumption of the model is that the data that are available for a given individual are representative of how that individual deviates from the population trend across the timeframe of the study. MRMs for longitudinal data using the maximum likelihood estimation provide valid statistical tests in the presence of ignorable nonresponse.

Note on Compound Symmetry and lntraclass Correlation:

Compund symetry is implied by the random intercept. If the covariance is expressed as a correlation, the intraclass correlation is derived. Individual variance over total variance

$$ICC = \frac{\sigma_{v}^2}{\sigma^2 + \sigma_{v}^{2}}$$ 

Hypothesis testing for fixed effects parameters ($\beta$'s) calls for Wald's test or Z-tests or chi-squared tests on one degree of freedom. However, for the Variance Covariance terms, Wald's test is nit statistically sound.

Liklihood ratio tests can be used on nested models for uni or multi parameter hypothesis tests.
